In [1]:
# Required imports.
import pandas as pd
from py import calc_ubi
from py.loss_functions import loss_metrics
from py.optimize import optimize

In [2]:
# Adult is calculated to achieve budget-neutrality; leave it off.
AGE_CATEGORIES = ["senior", "child"]
DIS_CATEGORIES = ["dis_base"]
REGIONS = [
    "NORTH_EAST",
    "NORTH_WEST",
    "YORKSHIRE",
    "EAST_MIDLANDS",
    "WEST_MIDLANDS",
    "EAST_OF_ENGLAND",
    "LONDON",
    "SOUTH_EAST",
    "SOUTH_WEST",
    "WALES",
    "SCOTLAND",
    "NORTHERN_IRELAND",
]
categories = AGE_CATEGORIES + DIS_CATEGORIES + REGIONS

# Define bounds
AGE_BOUNDS = (40, 240)  # Child, working-age adult, senior.
DIS_BOUNDS = (0, 200)  # Base, severe, and enhanced disability UBI supplements.
GEO_BOUNDS = (-50, 50)  # Will be relative to a baseline geo.
# Skip adult which is calculated.
bounds = ([AGE_BOUNDS] * len(AGE_CATEGORIES) +
          [DIS_BOUNDS] * len(DIS_CATEGORIES) +
          [GEO_BOUNDS] * len(REGIONS))

input_dict = {category: bound for category, bound in zip(categories, bounds)}
print(input_dict)

{'senior': (40, 240), 'child': (40, 240), 'dis_base': (0, 200), 'NORTH_EAST': (-50, 50), 'NORTH_WEST': (-50, 50), 'YORKSHIRE': (-50, 50), 'EAST_MIDLANDS': (-50, 50), 'WEST_MIDLANDS': (-50, 50), 'EAST_OF_ENGLAND': (-50, 50), 'LONDON': (-50, 50), 'SOUTH_EAST': (-50, 50), 'SOUTH_WEST': (-50, 50), 'WALES': (-50, 50), 'SCOTLAND': (-50, 50), 'NORTHERN_IRELAND': (-50, 50)}


In [3]:
def opt(reform):
    return optimize(input_dict, "mean_pct_loss", reform,
                    verbose=False, seed=0,
                    # Reforms don't always improve upon one another with the
                    # default tolerance of 0.01.
                    tol=0.0001)

In [4]:
%%time
reform_1 = opt("reform_1")
print(reform_1[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.514000e-01
losses                1.401361e+11
mean_pct_loss         5.190000e-02
mean_pct_loss_pwd2    5.830000e-02
gini                  2.749000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0519 

Optimal solution:
 adult               124
senior              205
child                76
dis_base              0
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
     fun: 0.05193292344448038
 message: 'Optimization terminated successfully.'
    nfev: 3182
     nit: 13
 success: True
       x: array([205.13710843,  75.56780549,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.     

In [5]:
%%time
reform_2 = opt("reform_2")
print(reform_2[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.559000e-01
losses                1.417501e+11
mean_pct_loss         4.980000e-02
mean_pct_loss_pwd2    5.140000e-02
gini                  2.711000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0498 

Optimal solution:
 adult               119
senior              196
child                76
dis_base             69
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
     fun: 0.04975748786214303
 message: 'Optimization terminated successfully.'
    nfev: 4082
     nit: 17
 success: True
       x: array([196.29118328,  76.06187817,  68.65539688,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.     

In [6]:
%%time
reform_3 = opt("reform_3")
print(reform_3[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.566000e-01
losses                1.412244e+11
mean_pct_loss         4.970000e-02
mean_pct_loss_pwd2    5.130000e-02
gini                  2.714000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0497 

Optimal solution:
 adult               115
senior              192
child                71
dis_base             69
NORTH_EAST            7
NORTH_WEST            6
YORKSHIRE             0
EAST_MIDLANDS         4
WEST_MIDLANDS         4
EAST_OF_ENGLAND       3
LONDON                8
SOUTH_EAST            4
SOUTH_WEST            6
WALES                 4
SCOTLAND              5
NORTHERN_IRELAND      4
dtype: int64
     fun: 0.04968460018692651
 message: 'Optimization terminated successfully.'
    nfev: 16007
     nit: 70
 success: True
       x: array([ 1.95267025e+02,  7.45729374e+01,  6.93519024e+01,  3.49023810e+00,
        2.52628836e+00, -3.57926583e+00,  8.47033278e-01,  8.21620033e-01,
       -5.56136694e-01,  4.28714465e+00,  6.35635926e-

Check that iterations improve.

In [7]:
assert reform_2[0].fun < reform_1[0].fun, "Reform 2 doesn't improve on 1"
assert reform_3[0].fun < reform_2[0].fun, "Reform 3 doesn't improve on 2"

Construct DataFrame to export, with parameter and optimization values.

In [8]:
ubi_params = pd.DataFrame(reform_1[1]).T
ubi_params.loc[1] = reform_2[1]
ubi_params.loc[2] = reform_3[1]
ubi_params['mean_pct_loss'] = [reform_1[0].fun,
                               reform_2[0].fun,
                               reform_3[0].fun]
ubi_params

,adult,senior,child,dis_base,NORTH_EAST,NORTH_WEST,YORKSHIRE,EAST_MIDLANDS,WEST_MIDLANDS,EAST_OF_ENGLAND,LONDON,SOUTH_EAST,SOUTH_WEST,WALES,SCOTLAND,NORTHERN_IRELAND,mean_pct_loss
0,123.956949,205.137108,75.567805,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.051933
1,119.466540,196.291183,76.061878,68.655397,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.049757
2,114.671299,191.687759,70.993672,69.351902,7.069504,6.105554,0.0,4.426299,4.400886,3.023129,7.86641,4.214902,5.996309,3.581901,5.09669,3.579266,0.049685


In [9]:
ubi_params.to_csv("optimal_params.csv", index=False)